# Train `Acceptability` model

## Setup and Data Loading

In [ ]:
!pip install transformers datasets tensorflow torch scikit-learn

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
import pandas as pd

In [3]:
dataset = load_dataset("glue", "cola")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [4]:
dataset['train'][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

## Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Model Definition and Training Arguments

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./roberta_results",
    per_device_train_batch_size=16,  # Adjust as needed
    per_device_eval_batch_size=64,
    num_train_epochs=3,  # Adjust as needed
    learning_rate=2e-5,  # Adjust learning rate
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="mcc",
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    mcc = matthews_corrcoef(labels, predictions)
    return {"accuracy": accuracy, "f1": f1, "mcc": mcc}

## Trainer and Training Loop

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Mcc
1,0.478800,0.421032,0.818792,0.878613,0.552066
2,0.279400,0.519015,0.824545,0.882164,0.567573
3,0.172300,0.673381,0.826462,0.882085,0.573077


TrainOutput(global_step=1605, training_loss=0.302469428231783, metrics={'train_runtime': 2535.2125, 'train_samples_per_second': 10.119, 'train_steps_per_second': 0.633, 'total_flos': 6749587903150080.0, 'train_loss': 0.302469428231783, 'epoch': 3.0})

## Evaluation

In [ ]:
best_model_checkpoint = trainer.state.best_model_checkpoint
best_model = AutoModelForSequenceClassification.from_pretrained(best_model_checkpoint)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
best_model.to(device)

In [ ]:
def predict_in_batches(model, dataset, batch_size=16):
    all_predictions = []
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i : i + batch_size]
        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        batch_input = {"input_ids": input_ids, "attention_mask": attention_mask}
        with torch.no_grad():
            outputs = model(**batch_input)
            logits = outputs.logits.to("cpu")
            predictions = logits.detach().numpy()
        all_predictions.append(predictions)
    return np.concatenate(all_predictions, axis=0)

predictions = predict_in_batches(best_model, tokenized_datasets["test"])
pred_labels = np.argmax(predictions, axis=-1)
true_labels = tokenized_datasets["test"]["label"]

test_metrics = compute_metrics((predictions, true_labels))
print("Test set metrics:", test_metrics)

# 5. Save the Best Model

In [ ]:
best_model.save_pretrained("./acceptability_model")

## 6.  Single Sentence Prediction Example

In [ ]:
def predict_single_sentence(sentence, model, tokenizer):
    inputs = tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class = logits.argmax().item()
    return "Acceptable" if predicted_class == 1 else "Unacceptable"


example_sentence = "Colorless green ideas sleep furiously."
prediction = predict_single_sentence(example_sentence, best_model, tokenizer)
print(f"'{example_sentence}' is predicted as: {prediction}")